In [1]:
from IPython.display import HTML, Image, display

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import table

Fair Policing Group 4
=================
### Stephanie Moore, Luis Diaz, Julian Yeh
##### Tasks (Member assigned):
* Cleaning + EDA on Arrests data (Stephanie) 
* Cleaning + EDA on Stops data (Luis)
* Cleaning + EDA on Crimes data (Julian)

# Cleaning & Exploratory Data Analysis
## Data Cleaning

### Crimes

In [ ]:
# Crimes data TODO

# Describe cleaning/processing steps/justifications

# Select relevant viz to show

### Stops

In [ ]:
# Stops Data TODO

# Don't need stop type, 2nd officer information, drop descent code/keep description, drop form reference number,

# Add column indicating whether reporting district is outside of officer's division (may indicate reassignment by PredPol
# if year falls under 2013-2019, and depends on Division as well since not all divisions starting using it at the same time)

# Join Reporting District with its Division (Call it Reporting Division) 
# -> Create new column "Reassigned" = stops['Division Description' or 'Division Number'] == stops['Reporting Division']

# Describe cleaning process/justification (Division, Reporting District)

# Select relevant graphs/tables/etc to include in checkpoint writeup

In [ ]:
stops_raw = pd.read_csv('../data/raw/stops.csv')
stops_raw.isna().mean().round(5)

In [7]:
stops_cleaned = pd.read_csv('../data/cleaned/stops-processed.csv')
stops_cleaned.isna().mean().round(5)

,0
Stop Number,0.0
Sex Code,0.0
Descent Description,0.0
Stop Date,0.0
Stop Time,0.0
Year,0.0
Officer 1 Serial Number,0.0
Officer 1 Division Number,0.0
Division Description 1,0.0
Reporting District,0.0


In [30]:
stops_cleaned.isna().mean().round(5).to_frame().reset_index().rename(columns={0:'Proportion of Null Values',
                                                                              'index':'Column Name'}).to_csv('test.csv', index=False)

In [31]:
pd.read_csv('test.csv')

,Column Name,Proportion of Null Values
0,Stop Number,0.0
1,Sex Code,0.0
2,Descent Description,0.0
3,Stop Date,0.0
4,Stop Time,0.0
5,Year,0.0
6,Officer 1 Serial Number,0.0
7,Officer 1 Division Number,0.0
8,Division Description 1,0.0
9,Reporting District,0.0


### Arrests

In [ ]:
# Arrests data TODO

# Waiting on census data for denominator to compute arrest rates

# Writeup + choosing relevant viz

# Revisions to proposal (if any)

# Backlog & Schedule (updated)